# How do I mount files from my own _cloud storage_?
### Overview
Connecting to cloud storage facilitates sharing and leveraging bioinformatics data that is commonly stored there (details [here](http://docs.sevenbridges.com/docs/connecting-cloud-storage-overview)). Here, we show an example of **reading** files from the _Seven Bridges Platform_ from a users _Amazon Web Services_ **bucket**. 

### Prerequisites
 1. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 2. You need to have an AWS account with an S3 bucket
 3. You have some files on this bucket, e.g. by following this [recipe](volumes_writeToCloudStorage.ipynb)
 3. You need to be a member (or owner) of a project, e.g. by following this [recipe](projects_makeNew.ipynb)
 
### Setting up Identity and Access Management (IAM) access 
There is a detailed [tutorial](http://docs.sevenbridges.com/v1.0/docs/aws-cloud-storage-tutorial) to set up access to your buckets. Please follow that tutorials and make note of your **policy name(s)**. Also make sure to generate an _secret access key_.

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below. We also define<sup>1</sup> a function to wrap the Volumes API calls.

<sup>1</sup> Support for _Volumes API_ is on the immediate roadmap for the _sevenbridges-python_ library, once it is implemented the api_call() function is unnecessary.

In [ ]:
import sevenbridges as sbg

import json
from requests import request


def api_call(path, method='GET', query=None, data=None, token=None):
    
    base_url = 'https://api.sbgenomics.com/v2/storage/'
    
    data = json.dumps(data) if isinstance(data, dict) \
    or isinstance(data,list) else None
              
    headers = {
        'X-SBG-Auth-Token': token,
        'Accept': 'application/json',
        'Content-type': 'application/json',
    }
    
    response = request(method, base_url + path, params=query, \
                       data=data, headers=headers) 
    response_dict = json.loads(response.content) if \
    response.content else {}
    
    if response.status_code / 100 != 2:
        print(response_dict['message'])
        print('Error Code: %i.' % (response_dict['code']))
        print(response_dict['more_info'])
        raise Exception('Server responded with status code %s.' \
                        % response.status_code)
    return response_dict

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Mount the volume
Since we have configured the IAM policy in the prior step, we can now mount the volume.

#### Additional information
 * Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-volume-v2)

In [ ]:
body = {
  "name": "my_volume_RO",
  "service": {
    "type": "s3",
    "bucket": "BUCKET-NAME",
    "prefix": "",
    "credentials": {
      "access_key_id": "INSERT ACCESS KEY ID",
      "secret_access_key": "INSERT SECRET ACCESS KEY"
    },
    "properties": {
      "sse_algorithm": "AES256"
    }
  },
  "access_mode": "RO"
}

response = api_call(path = 'volumes/', method = 'POST', \
                    token = config_file.auth_token, data = body)

## Get details of the volume
Let's list all our volumes and get one volume's details to make sure it's mounted as we expected.

### Warning
We've hardcoded the single_volume to detail, this could easily be selected by matching 'name' instead

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here (list)](http://docs.sevenbridges.com/docs/list-volumes-v2) and [here (details)](http://docs.sevenbridges.com/docs/get-details-of-a-volume-v2)

In [ ]:
my_volumes = api_call(path = 'volumes', method = 'GET', \
                    token = config_file.auth_token)
for v in my_volumes['items']:
    print(v)
    

single_volume = api_call(path = ('volumes/' + my_volumes['items'][1]['id']), \
                         method = 'GET', token = config_file.auth_token)
print(single_volume)

## Set up a new project
We will first make an empty project, verify that there are no files in it, then mount files there.


#### PROTIPS
 * A detailed _recipe_ for creating projects is [here](projects_makeNew.ipynb)
 * A detailed _recipe_ for listing files is [here](files_listAll.ipynb)

In [ ]:
# [USER INPUT] Set project name here:
new_project_name = 'MCA'  


# What are my funding sources?
billing_groups = api.billing_groups.query()  

# Set up the information for your new project
new_project = {
        'billing_group': billing_groups[0].id,
        'description': 
    """A project created by the API recipe (volumes_readFromCloudStorage.ipynb).
    This also supports **markdown**
    _Pretty cool_, right?
    """,
        'name': new_project_name
}

# check if this project already exists. LIST all projects and check for name match
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == new_project_name]      
              
if my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('A project with the name (%s) already exists, please pick a new one' \
          % new_project_name)
    raise KeyboardInterrupt
else:
    # CREATE the new project
    my_project = api.projects.create(name = new_project['name'], \
                                     billing_group = new_project['billing_group'], \
                                     description = new_project['description'], \
                                     )
    
    # (re)list all projects, and get your new project
    my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == new_project_name][0]

    print('Your new project %s has been created.' % (my_project.name))
    if hasattr(my_project, 'description'): # need to check if description has been entered
        print('Project description: %s \n' % (my_project.description)) 
        
        
my_files = api.files.query(limit=100, project = my_project)
if my_files.total == 0:
    print('no files in the project')

## Start an import job
Spin up a job to import files to the _Seven Bridges Platform_ from your _Cloud Storage_ (Volume).

### WARNING
The file names here are hardcoded (here empty) - please adjust for your own use case.

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/start-an-export-job-v2)

In [ ]:
# [USER INPUT] Set files to import from your volume here:
file_list = ['',
     '', 
     ''
     ]

# Loop through selected files, start one job for each.
for f_name in file_list:
    body = {
        "source":{
            "volume": single_volume['id'],
            "location": f_name
        },
        "destination":{  
            "project":my_project.id,
            "name": f_name
        },
        "overwrite": True
    }   
    
    response = api_call(path = 'imports', method = 'POST', \
                    token = config_file.auth_token, data = body)

## List and detail import jobs
The first call (list) is sufficient to keep track of the job progress in 

```python
    import_jobs['items'][0]['state']
```

The second call reveals more detail about the job.

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/list-import-jobs-v2) and [here](http://docs.sevenbridges.com/docs/get-details-of-an-import-job-v2)

In [ ]:
import_jobs = api_call(path = 'imports', method = 'GET', \
                    token = config_file.auth_token)

print("Example of information available in the list call:")
print(import_jobs['items'][0])

job_details = api_call(path = ('imports/' + import_jobs['items'][0]['id']), \
                      method = 'GET', token = config_file.auth_token)

print("\n Example of information available in the detail call:")
print(job_details)